In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from astropy.stats.circstats import circmean
from functools import reduce
import datetime
import time
import plotly.express as px
import numpy as np

pd.set_option("display.precision", 2)
plt.rcParams.update({'font.size': 20, 'figure.figsize': (8, 4)})

%matplotlib inline
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
working_df = pd.read_csv("measurements_full.csv")
working_df['numeric_id'] = working_df['id'].apply(lambda id: int(id))
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3027 entries, 0 to 3026
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3027 non-null   int64  
 1   date               3027 non-null   object 
 2   steps              3027 non-null   int64  
 3   average heartrate  3027 non-null   int64  
 4   average stress     3027 non-null   int64  
 5   sleep time         2423 non-null   object 
 6   duration           2423 non-null   float64
 7   awake time         2423 non-null   object 
 8   start_seconds      2423 non-null   float64
 9   duration_seconds   2423 non-null   float64
 10  waking_seconds     2423 non-null   float64
 11  numeric_id         3027 non-null   int64  
dtypes: float64(4), int64(5), object(3)
memory usage: 283.9+ KB


### adding sleeping_hour column to dataframe

In [3]:
def convert_epoch_to_time(epoch):
    if not pd.isna(epoch):
        return time.strftime('%H:%M', time.localtime(epoch))

    
working_df['sleeping_hour'] = working_df['start_seconds'].apply(convert_epoch_to_time)
working_df

id        date  steps  average heartrate  average stress  \
0     101  21/01/2023   4334                121              -1   
1     101  22/01/2023   1753                104              26   
2     101  23/01/2023   2136                 89              19   
3     101  24/01/2023   1360                 76              27   
4     101  25/01/2023   5275                 74              27   
...   ...         ...    ...                ...             ...   
3022  229  16/03/2023  18362                 87              46   
3023  229  17/03/2023   1854                 77              27   
3024  229  18/03/2023   4079                 77              32   
3025  229  19/03/2023    193                 65               7   
3026  229  20/03/2023  12817                 88              45   

            sleep time  duration awake time  start_seconds  duration_seconds  \
0                  NaN       NaN        NaN            NaN               NaN   
1                  NaN       NaN        NaN            NaN               NaN   
2                  NaN       NaN        NaN            NaN               NaN   
3                  NaN       NaN        NaN            NaN               NaN   
4     24/01/2023 23:02      7.13       6:10       1.67e+09           25680.0   
...                ...       ...        ...            ...               ...   
3022  15/03/2023 19:30      5.58       1:05       1.68e+09           20100.0   
3023  16/03/2023 23:04      4.42       3:29       1.68e+09           15900.0   
3024   18/03/2023 2:13      7.08       9:18       1.68e+09           25500.0   
3025   19/03/2023 0:30      7.00       7:30       1.68e+09           25200.0   
3026               NaN       NaN        NaN            NaN               NaN   

      waking_seconds  numeric_id sleeping_hour  
0                NaN         101          None  
1                NaN         101          None  
2                NaN         101          None  
3                NaN         101          None  
4           1.67e+09         101         23:02  
...              ...         ...           ...  
3022        1.68e+09         229         19:30  
3023        1.68e+09         229         23:04  
3024        1.68e+09         229         02:13  
3025        1.68e+09         229         00:30  
3026             NaN         229          None  

[3027 rows x 13 columns]

### creating main table that includes [id, date, heartRate, stress, steps, sleeping_time, sleeping_duration, waking_time, midSleep, week-end, exams]

In [4]:
def is_weekend(date):
    """
    input: a string that descripe a date in format {day}/{month}/{year}
    output: - "Yes" if the date is weekend (friday/saturday)
            - "No" otherwise
    """
    date_arr = date.split('/')
    day, month, year = int(date_arr[0]), int(date_arr[1]), int(date_arr[2])
    date_time = datetime.datetime(year, month, day)
    
    if (date_time.strftime('%A') == 'Friday' or date_time.strftime('%A') == 'Saturday'):
        return 'Yes'
    else:
        return 'No'


def is_exams_period(date):
    """
    input: a string that descripe a date in format {day}/{month}/{year}
    output: - "Yes" if the date falls in exams perioud (25/1/2023 - 15/2/2023)
            - "No" otherwise
    """
    date_arr = date.split('/')
    day, month, year = int(date_arr[0]), int(date_arr[1]), int(date_arr[2])
    
    if (month == 1 and day >= 25) or (month == 2 and day <= 15):
        return 'Yes'
    else:
        return 'No'

    
def convert_epoch_to_time(epoch_time):
    """
    input : epoch time
    output : a string that describe the time in format {hours}:{minutes}
    """
    if not pd.isna(epoch_time):
        return datetime.datetime.fromtimestamp(epoch_time).strftime('%H:%M')


working_df['weekend'] = working_df['date'].apply(is_weekend)
working_df['exam period'] = working_df['date'].apply(is_exams_period)
working_df['midsleep_seconds'] = working_df['start_seconds'] + (working_df['duration_seconds'] / 2)
working_df['midSleep time'] = working_df['midsleep_seconds'].apply(convert_epoch_to_time)
working_df = working_df.fillna(value=np.nan) # replace None values with NaN values
#deleting unwanted columns (that researches doesnt need) from dataframe
measurements = working_df.drop(['start_seconds', 'duration_seconds', 'waking_seconds', 'numeric_id', 'midsleep_seconds', 'sleeping_hour'], axis=1)
#re-ordering columns
measurements = measurements[['id','date', 'steps', 'average heartrate', 'average stress', 'sleep time', 'duration', 'awake time', 'midSleep time', 'weekend', 'exam period']]

### save measurements dataframe to csv

In [5]:
measurements.to_csv('measurements.csv', index=False)
measurements

id        date  steps  average heartrate  average stress  \
0     101  21/01/2023   4334                121              -1   
1     101  22/01/2023   1753                104              26   
2     101  23/01/2023   2136                 89              19   
3     101  24/01/2023   1360                 76              27   
4     101  25/01/2023   5275                 74              27   
...   ...         ...    ...                ...             ...   
3022  229  16/03/2023  18362                 87              46   
3023  229  17/03/2023   1854                 77              27   
3024  229  18/03/2023   4079                 77              32   
3025  229  19/03/2023    193                 65               7   
3026  229  20/03/2023  12817                 88              45   

            sleep time  duration awake time midSleep time weekend exam period  
0                  NaN       NaN        NaN           NaN     Yes          No  
1                  NaN       NaN        NaN           NaN      No          No  
2                  NaN       NaN        NaN           NaN      No          No  
3                  NaN       NaN        NaN           NaN      No          No  
4     24/01/2023 23:02      7.13       6:10         02:36      No         Yes  
...                ...       ...        ...           ...     ...         ...  
3022  15/03/2023 19:30      5.58       1:05         22:17      No          No  
3023  16/03/2023 23:04      4.42       3:29         01:16     Yes          No  
3024   18/03/2023 2:13      7.08       9:18         05:45     Yes          No  
3025   19/03/2023 0:30      7.00       7:30         04:00      No          No  
3026               NaN       NaN        NaN           NaN      No          No  

[3027 rows x 11 columns]

### coding a function that recieve dataframe and returns averages dataframe

In [6]:
def create_avg_time_df(time_df, time_column_name, average_column_name):
    """
    the function recieves dataframe with time columns, it returns average time dataframe grouped by id
    """
    time_df["radians"] = pd.to_datetime(time_df[time_column_name], format="%H:%M").dt.hour / 24 * 2 * np.pi
    
    # Compute the circular mean of the radians for each group
    average_df = time_df.groupby("id").agg({"radians": circmean, time_column_name: "first"})
    #deleting radians column to not effect recieved dataframe
    time_df.drop(['radians'], axis=1, inplace = True)
    # Convert the radians to "hour:minute" format
    average_df[average_column_name] = pd.to_datetime(average_df["radians"] / (2 * np.pi) * 24, unit="h").dt.strftime("%H:%M")
    
    return average_df.drop(['radians',time_column_name], axis=1)


def create_avg_table_from_df(measurments_df):
    """
    input: a final research results dataframe 
    output: averages dataframe that include the following average columns => 
     => [steps, average heartrate, average stress, average sleep start, duration, average awake time, average midsleep ]         
    """
    ######################calculating average for steps predictors##########################
    
    #creating df without 0 heartrate value rows
    clean_heart_rate_df = measurments_df[measurments_df['average heartrate'] != 0]
    #creating df without -1 stress level rows
    clean_stress_level_df = measurments_df[measurments_df['average stress'] != -1]

    #average dailies predictors dataframe creation
    average_steps_df = measurments_df.groupby('id')['steps'].mean().to_frame()
    #average heartrate dataframe creation
    average_heartRate_df = clean_heart_rate_df.groupby('id')['average heartrate'].mean().to_frame()
    #average stress level dataframe creation
    average_stress_df = clean_stress_level_df.groupby('id')['average stress'].mean().to_frame()


    #merging all dailies average dataframes into one dataframe
    dailies_df_list = [average_steps_df, average_heartRate_df, average_stress_df]
    merged_dailies_df = reduce(lambda left, right: pd.merge(left, right, on="id", how="inner"), dailies_df_list)
    ######################calculating average for sleeps predictors############################
    
    #removing empty sleeps rows
    clean_sleeps_df = measurments_df.dropna()
    
    #average sleeps predictors dataframe creation
    average_sleep_df = create_avg_time_df(clean_sleeps_df, 'sleeping_hour','average sleep start')
    average_sleep_duration_df = clean_sleeps_df.groupby('id')['duration'].mean().to_frame()
    average_awake_df = create_avg_time_df(clean_sleeps_df, 'awake time','average awake time')
    average_midsleep_df = create_avg_time_df(clean_sleeps_df, 'midSleep time','average midsleep')
    
    # merging all sleeps average dataframes into one dataframe
    sleeping_df_list = [average_sleep_df, average_sleep_duration_df, average_midsleep_df, average_awake_df]
    merged_sleeping_df = reduce(lambda left, right: pd.merge(left, right, on="id", how="inner"), sleeping_df_list)

    #creating final merged df
    merged_df = pd.merge(merged_dailies_df, merged_sleeping_df, on='id', how='left')
    
    return merged_df


### creating morning-type, exams-period, mid-week averages table

In [7]:
morning_exams_midweek_students_df = working_df[(working_df['numeric_id'] >= 200) & (working_df['weekend']=='No') & (working_df['exam period']=='Yes')]
morning_exams_midweek_average_df = create_avg_table_from_df(morning_exams_midweek_students_df)

### saving morning-type, mid-week, exams-duration averages dataframe to csv

In [8]:
morning_exams_midweek_average_df.to_csv('morning-midweek-exams-averages.csv')
morning_exams_midweek_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
201  16022.13              73.13           28.60               22:24   
202   2875.19              74.06           26.25               22:52   
203   2684.14              87.14           37.21               21:53   
204   3079.43              71.50           30.07               23:24   
205   4311.53              78.93           33.80               22:43   
206   7304.56              83.25           31.94               23:36   
207   6124.21              81.00           25.50               23:21   
208   3300.06              85.88           31.38               23:37   
209   3561.75              81.94           39.19               22:17   
210   2986.75              78.12           27.38               00:03   
211   5536.86              81.46           23.23               23:07   
213   4556.38              82.69           27.81               22:18   
214   2670.12              88.12           33.94               23:51   
215   3646.00              89.75           43.12               00:38   
216   5902.33              83.42           42.25               23:43   
217   6210.60              77.33           30.40               23:44   
218   1888.33              66.00           25.33               23:00   
219   4098.50              81.79           33.43               22:41   
220   4152.41              85.82           32.71               22:19   
221   2338.75              76.25           32.81               23:35   
222   8057.69              83.69           28.46               23:24   
223   1959.38              84.38           53.50               20:32   
224   5827.69              80.75           30.12               22:33   
225   8597.44              78.69           22.88               22:52   
227   4951.50              72.44           31.44               00:04   
228  10100.47              76.80           30.73               23:19   
229   4954.00              84.71           40.14               23:19   

     duration average midsleep average awake time  
id                                                 
201      9.26            02:54              07:35  
202      8.17            02:59              07:22  
203      7.33            01:19              05:00  
204      8.46            03:36              07:48  
205      8.71            03:11              07:22  
206      7.45            03:16              06:52  
207      7.16            03:05              06:49  
208      8.81            03:56              08:24  
209      8.31            02:22              06:35  
210      6.49            03:26              06:55  
211      7.81            03:15              07:15  
213      9.16            03:11              07:45  
214      7.77            03:38              07:33  
215      6.23            03:45              06:58  
216      7.23            03:21              06:59  
217      7.18            03:15              06:50  
218      8.52            03:00              08:00  
219      7.77            02:22              06:27  
220      7.69            02:06              06:06  
221      7.80            03:12              07:24  
222      7.42            03:00              06:41  
223      9.19            01:03              05:41  
224      7.84            02:21              06:19  
225      8.02            02:56              07:00  
227      8.40            04:16              08:24  
228      7.33            03:00              06:45  
229      7.65            03:15              06:59

### creating morning-type, exams-period, weekend averages table

In [9]:
morning_exams_weekend_students_df = working_df[(working_df['numeric_id'] >= 200) & (working_df['weekend']=='Yes') & (working_df['exam period']=='Yes')]
morning_exams_weekend_average_df = create_avg_table_from_df(morning_exams_weekend_students_df)

### saving morning-type, exams-period, weekend averages dataframe to csv

In [10]:
morning_exams_weekend_average_df.to_csv('morning-weekend-exams-averages.csv')
morning_exams_weekend_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
201  10694.33              70.50           30.00               23:45   
202   2124.50              73.00           23.33               23:39   
203   3042.50              87.50           33.33               22:00   
204   5088.00              71.50           26.83               23:00   
205   6557.00              77.00           24.20               22:44   
206   7931.00              82.17           28.67               23:45   
207   4578.17              79.17           28.17               00:00   
208   2091.50              84.00           30.00               23:43   
209   6329.67              82.00           33.67               00:11   
210   3469.00              76.17           24.17               00:00   
211   6167.80              79.60           16.80               23:14   
213   4156.83              81.50           28.17               22:49   
214   3567.33              89.83           31.33               00:20   
215   2969.33              88.33           37.00               00:15   
216   7980.75              86.75           45.00               23:30   
217   7083.00              78.17           30.17               00:12   
218   3724.00              63.67           27.33               23:00   
219   2489.67              79.83           33.17               23:15   
220   5931.00              83.83           27.83               23:00   
221   2763.33              72.83           29.33               22:00   
222   9722.33              80.00           21.33               00:20   
223   2218.50              89.17           58.67               20:58   
224   8588.50              81.67           30.00               23:00   
225   8458.83              81.00           24.00               00:25   
227   3760.17              70.67           29.67               00:22   
228   5689.00              73.50           27.33               22:47   
229   3499.50              82.67           33.67               23:40   

     duration average midsleep average awake time  
id                                                 
201      9.69            04:15              09:14  
202      7.73            03:49              07:40  
203      7.24            01:19              05:00  
204      9.14            03:29              08:00  
205      8.84            03:00              07:30  
206      7.57            03:52              07:40  
207      8.04            04:12              08:00  
208      8.38            03:45              08:01  
209      7.51            03:51              07:40  
210      7.08            03:35              07:23  
211      7.92            03:14              07:45  
213      9.51            03:29              08:19  
214      7.96            04:10              08:20  
215      8.34            04:36              08:36  
216      6.78            03:00              06:00  
217      7.52            04:00              07:36  
218      8.43            03:00              07:00  
219      8.14            03:15              07:15  
220      6.39            02:00              05:16  
221      9.10            02:49              07:30  
222      6.96            03:40              07:19  
223      9.35            01:29              06:10  
224      8.05            03:00              07:00  
225      7.26            04:36              07:48  
227      8.59            04:47              09:11  
228      7.48            02:35              06:36  
229      7.95            03:40              07:40

### creating morning-type, non-exams, weekend average table

In [11]:
morning_non_exams_weekend_students_df = working_df[(working_df['numeric_id'] >= 200) & (working_df['weekend']=='Yes') & (working_df['exam period']=='No')]
morning_non_exams_weekend_averages_df = create_avg_table_from_df(morning_non_exams_weekend_students_df)

### saving morning-type, non-exams, weekend averages dataframe to csv

In [12]:
morning_non_exams_weekend_averages_df.to_csv('morning-weekend-non-exams-averages.csv')
morning_non_exams_weekend_averages_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
201  10195.55              67.27           22.27               22:33   
202   4407.62              75.46           19.77               23:11   
203   7583.08              85.08           28.17               22:10   
204   6212.80              72.00           32.20                 NaN   
205   5897.60              78.20           23.60               23:14   
206  11980.45              88.00           34.55               00:45   
207   8854.82              82.64           27.36               01:25   
208   5883.00              88.00           29.73               23:53   
209   8044.73              82.60           32.70               22:24   
210   5542.73              79.45           26.73               00:33   
211   9123.75              92.50           31.27               22:19   
213   6676.45              83.27           27.36               23:41   
214   3827.83              89.00           31.00               23:49   
215   3335.00              83.64           30.18               23:23   
216   9957.42              84.83           43.25               01:00   
217  13570.18              82.73           33.82               23:54   
218   6575.00              62.50           20.25               22:44   
219   9144.18              78.91           24.55               23:12   
220   9597.00              86.67           27.33               00:29   
221   3064.64              74.64           30.55               23:17   
222   7610.88              78.50           28.75               22:15   
223   2957.50              90.92           45.42               21:23   
224   8104.00              86.29           33.14               22:39   
225  16577.75              82.83           24.25               01:13   
227   4795.25              69.83           27.83               00:55   
228  13805.73              82.45           27.18               02:00   
229   5611.89              80.00           26.38               23:58   

     duration average midsleep average awake time  
id                                                 
201     11.47            04:20              10:14  
202      9.35            04:05              08:44  
203      8.26            02:23              06:30  
204       NaN              NaN                NaN  
205      8.68            03:42              08:05  
206      7.39            04:22              08:07  
207      7.73            05:12              09:11  
208      9.52            04:33              09:25  
209      9.15            03:05              07:46  
210      6.83            03:53              07:26  
211      8.32            03:00              07:00  
213      8.80            04:16              08:32  
214      8.72            04:11              08:39  
215      8.79            03:47              08:13  
216      7.70            05:00              08:48  
217      6.43            03:11              06:17  
218      9.40            02:58              07:30  
219      8.30            03:24              07:42  
220      7.72            04:00              07:30  
221      8.31            03:22              07:39  
222      8.82            02:30              06:45  
223      9.09            01:44              06:30  
224      8.42            02:39              06:58  
225      6.99            04:59              08:20  
227      8.43            05:06              09:17  
228      7.34            05:40              09:20  
229      7.09            03:31              07:05

### creating morning-type, non-exams, midweek average table

In [13]:
morning_non_exams_midweek_students_df = working_df[(working_df['numeric_id'] >= 200) & (working_df['weekend']=='No') & (working_df['exam period']=='No')]
morning_non_exams_midweek_average_df = create_avg_table_from_df(morning_non_exams_midweek_students_df)

### saving morning-type, non-exams, weekend averages dataframe to csv

In [14]:
morning_non_exams_midweek_average_df.to_csv('morning-midweek-non-exams-averages.csv')
morning_non_exams_midweek_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
201  14101.76              70.31           27.07               22:31   
202   4212.20              75.10           22.03               23:06   
203   8332.41              88.07           33.52               22:28   
204   6125.90              75.35           38.00                 NaN   
205   8278.96              80.56           30.70               23:01   
206  10717.39              82.29           27.32               23:49   
207   9813.81              80.67           23.48               00:07   
208   5070.74              86.87           29.35               23:43   
209   8666.11              79.89           28.85               23:36   
210   6568.36              76.64           22.50               23:40   
211   7763.67              83.83           25.18               23:05   
213   7117.00              84.39           29.21               23:00   
214   3866.43              88.29           32.79               23:25   
215   4941.66              89.28           42.52               23:20   
216   8415.12              82.28           38.76               00:33   
217  12031.70              79.81           30.70               23:49   
218   7323.13              61.50           23.00               22:10   
219   9315.80              78.13           25.23               22:47   
220   5896.52              87.11           30.00               22:18   
221   5456.43              75.07           32.63               23:12   
222   7992.07              77.96           26.76               22:34   
223   3359.97              94.34           49.24               20:44   
224   9071.33              84.39           28.17               23:04   
225  14570.72              80.32           21.40               23:10   
227   6432.73              70.97           30.00               23:29   
228  11852.61              78.50           25.29               00:22   
229   6519.52              80.79           32.54               22:33   

     duration average midsleep average awake time  
id                                                 
201      9.59            03:14              08:13  
202      7.90            03:01              07:10  
203      7.63            02:16              06:14  
204       NaN              NaN                NaN  
205      8.30            03:10              07:23  
206      7.01            03:25              06:57  
207      6.86            03:26              06:52  
208      8.16            03:45              07:58  
209      7.83            03:25              07:18  
210      7.07            03:15              06:52  
211      7.30            02:43              06:38  
213      8.90            03:29              07:54  
214      6.79            02:50              06:11  
215      6.61            02:36              05:49  
216      7.63            04:23              08:10  
217      6.32            03:04              06:19  
218      9.35            02:54              07:37  
219      8.07            02:55              06:53  
220      8.35            02:22              06:27  
221      7.56            03:01              06:49  
222      7.82            02:31              06:22  
223      8.87            01:08              05:35  
224      7.93            03:18              07:09  
225      7.58            02:58              07:01  
227      8.36            03:49              07:59  
228      7.17            03:55              07:32  
229      7.95            02:31              06:40

### creating night-type, non-exams, midweek average table

In [15]:
night_non_exams_midweek_students_df = working_df[(working_df['numeric_id'] < 200) & (working_df['weekend']=='No') & (working_df['exam period']=='No')]
night_non_exams_midweek_average_df = create_avg_table_from_df(night_non_exams_midweek_students_df)

### saving night-type, non-exams, weekend averages dataframe to csv

In [16]:
night_non_exams_midweek_average_df.to_csv('night-midweek-non-exams-averages.csv')
night_non_exams_midweek_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
101   6443.00              74.75           28.65               23:05   
102   7405.00              96.30           44.22                 NaN   
105   8467.72              78.21           41.38               00:36   
107   6273.81              79.03           33.62               00:29   
108   5817.29              81.50           24.65               23:49   
109   6485.28              86.38           36.06               00:34   
110   3491.12              77.93           28.20               03:24   
111   7499.73              72.27           35.47               23:50   
112   9446.44              94.19           40.06               00:59   
113   5162.97              74.49           39.08               00:56   
115   8522.32              83.94           26.16               00:59   
116   6669.73              74.07           27.17               00:40   
117   4255.00              76.96           21.56               23:46   
118   4895.70              75.23           32.10               00:19   
119  16951.41              74.24           31.21               01:23   
121   6018.33              78.90           35.10               00:10   
122   9783.92              81.17           31.25               23:36   
123   8658.45              61.17           21.24               01:06   
124  12112.19              68.19           26.78               01:51   
125   4448.30              80.17           30.90               00:20   
126   5880.19              88.22           35.52               00:15   
127   7833.10              83.25           30.90               00:05   
129  10622.04              83.67           31.58               01:00   
130   7244.61              87.32           30.36               23:07   
131   5969.65              79.00           24.00               01:54   

     duration average midsleep average awake time  
id                                                 
101      8.38            03:16              07:29  
102       NaN              NaN                NaN  
105      8.84            05:05              09:27  
107     10.29            05:31              10:35  
108      7.95            03:51              07:43  
109      7.74            04:40              08:35  
110      9.83            08:17              13:21  
111      9.21            04:24              08:58  
112      8.11            05:17              09:11  
113      7.93            04:53              08:46  
115      7.51            04:39              08:21  
116      8.79            05:06              09:31  
117      8.05            03:52              07:48  
118      7.90            04:04              08:12  
119      7.22            05:03              08:41  
121      8.35            04:24              08:35  
122      8.14            03:43              07:46  
123      8.72            05:25              09:53  
124      7.68            05:41              09:31  
125      7.72            04:12              08:06  
126      8.72            04:41              08:57  
127      8.08            04:13              08:08  
129      8.14            05:09              09:18  
130      9.14            03:41              08:11  
131      9.57            06:42              11:35

### creating night-type, exams, midweek average table

In [17]:
night_exams_midweek_students_df = working_df[(working_df['numeric_id'] < 200) & (working_df['weekend']=='No') & (working_df['exam period']=='Yes')]
night_exams_midweek_average_df = create_avg_table_from_df(night_exams_midweek_students_df)

### saving night-type, exams, midweek averages dataframe to csv

In [18]:
night_exams_midweek_average_df.to_csv('night-midweek-exams-averages.csv')
night_exams_midweek_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
101   4958.88              72.31           29.00               22:37   
102   4499.13              92.07           42.07                 NaN   
105   6978.81              77.00           37.31               00:33   
107   5808.81              80.88           37.44               23:33   
108   5286.43              81.50           25.50               00:56   
109   4258.13              87.13           40.07               00:01   
110   3345.00              80.50           30.50               02:00   
111   6582.12              70.94           32.19               23:42   
112   3962.75              93.25           49.50               00:51   
113   4879.00              77.50           41.50               01:30   
115   4197.94              86.19           36.31               00:14   
116   7625.00              75.88           27.56               00:15   
117   2945.31              74.69           21.31               00:39   
118   3201.06              78.94           42.38               23:40   
119  12843.94              69.88           31.12               01:07   
121   3176.88              74.81           30.50               00:19   
122   5984.71              81.36           37.64               23:51   
123   7498.75              59.88           20.56               00:00   
124   9767.00              65.94           25.06               01:37   
125   4155.06              80.38           29.50               00:03   
126   4067.50              81.31           36.06               00:19   
127   1556.94              82.06           35.38               00:09   
129   9147.86              84.86           33.64               01:40   
130   2472.79              88.86           32.29               23:10   
131   3231.60              82.30           35.90               01:45   

     duration average midsleep average awake time  
id                                                 
101      8.98            03:03              07:29  
102       NaN              NaN                NaN  
105      8.86            05:03              09:29  
107      9.90            04:39              09:41  
108      8.26            05:01              09:16  
109      6.66            03:16              06:39  
110      6.83            05:30              08:28  
111      9.61            04:29              09:04  
112      7.71            04:42              08:16  
113      7.38            05:12              08:55  
115      8.02            04:07              08:16  
116      9.65            05:04              09:57  
117      7.85            04:32              08:33  
118      8.37            03:51              08:04  
119      7.44            04:52              08:41  
121      9.00            04:49              09:19  
122      7.16            03:27              07:00  
123      8.94            04:22              08:56  
124      8.02            05:30              09:34  
125      8.11            04:04              08:14  
126      9.45            05:08              09:42  
127      8.22            04:09              08:27  
129      6.95            05:15              08:46  
130      8.97            03:30              08:06  
131      9.20            06:45              11:14

### creating night-type, non-exams, weekend average table

In [19]:
night_non_exams_weekend_students_df = working_df[(working_df['numeric_id'] < 200) & (working_df['weekend']=='Yes') & (working_df['exam period']=='No')]
night_non_exams_weekend_average_df = create_avg_table_from_df(night_non_exams_weekend_students_df)

### saving night-type, non-exams, weekend averages dataframe to csv

In [20]:
night_non_exams_weekend_average_df.to_csv('night-weekend-non-exams-averages.csv')
night_non_exams_weekend_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
101   3860.24              74.44           22.07               23:02   
102  11666.90              95.00           36.67                 NaN   
105   9896.92              77.75           40.58               00:19   
107   6754.19              82.06           34.62               00:28   
108   8821.89              85.56           27.44               01:30   
109  11430.67              89.50           36.00               01:30   
110   8401.57              86.00           40.71               01:37   
111  11871.73              75.18           36.73               00:16   
112   8044.43              96.00           45.57               03:00   
113   4707.64              75.38           38.46               01:59   
115  10553.64              87.91           29.91               02:42   
116   6401.08              76.00           25.15               02:10   
117   4367.60              76.30           21.20               02:03   
118   4609.55              76.00           32.45               00:38   
119  17879.58              73.00           27.50               02:35   
121   8225.83              83.25           35.33               00:33   
122  10237.30              82.30           31.90               00:08   
123   8600.36              61.09           22.64               01:52   
124  11686.91              68.09           24.09               01:27   
125   3923.36              79.18           28.36               00:32   
126   2792.00              83.00           28.71               01:30   
127   7802.22              85.56           35.89               00:29   
129  13689.67              91.75           40.50               03:07   
130   3809.30              83.20           28.80               21:45   
131  10950.75              85.50           34.00               04:00   

     duration average midsleep average awake time  
id                                                 
101      9.09            03:41              07:55  
102       NaN              NaN                NaN  
105      8.67            04:50              09:00  
107     11.18            06:03              11:40  
108      8.49            05:45              09:30  
109      7.71            05:00              09:00  
110      7.69            05:12              08:57  
111      8.42            04:22              08:32  
112      7.88            07:02              10:40  
113      8.78            06:23              10:58  
115      6.78            06:15              09:44  
116      9.15            06:41              11:09  
117      7.88            06:10              09:52  
118      8.34            04:49              09:01  
119      7.32            06:15              09:56  
121      7.55            04:09              07:58  
122      8.31            04:23              08:29  
123      8.10            05:47              10:05  
124      8.10            05:20              09:26  
125      8.48            04:45              09:12  
126      8.67            05:40              10:10  
127      8.51            04:40              09:00  
129      7.19            06:49              10:23  
130     11.18            03:22              09:07  
131      9.76            09:13              14:02

### creating night-type, exams, weekend average table

In [21]:
night_exams_weekend_students_df = working_df[(working_df['numeric_id'] < 200) & (working_df['weekend']=='Yes') & (working_df['exam period']=='Yes')]
night_exams_weekend_average_df = create_avg_table_from_df(night_exams_weekend_students_df)

### saving night-type, exams, weekend averages dataframe to csv

In [22]:
night_exams_weekend_average_df.to_csv('night-weekend-exams-averages.csv')
night_exams_weekend_average_df

steps  average heartrate  average stress average sleep start  \
id                                                                     
101   3392.33              66.33           18.50               21:12   
102   8260.80              91.40           34.80               22:00   
105   5422.17              76.67           34.83               00:28   
107   5502.50              78.00           31.75               02:00   
108   4184.00              82.00           24.00               02:00   
109   5852.00              89.40           39.60               00:27   
110   5292.00              86.75           44.50               02:00   
111   8595.83              72.17           32.17               00:20   
112   8999.00              95.75           35.50               02:32   
113   4158.00              74.67           34.83               01:19   
115   4970.17              83.17           27.00               01:10   
116   6801.33              74.33           26.50               01:36   
117   3590.00              77.33           25.67               02:16   
118   3196.33              76.33           35.00               23:14   
119  11923.67              68.33           27.33               02:10   
121   2950.20              75.80           29.00               00:45   
122   8210.17              80.50           35.17               00:23   
123   8449.50              58.17           18.50               00:20   
124   7492.00              65.50           24.00               01:29   
125   3865.33              79.50           28.00               00:18   
126   2772.33              80.67           30.00               00:50   
127   4272.80              84.60           41.00               00:30   
129   9644.40              83.40           28.60               03:00   
130   1549.00              85.40           40.60               23:20   
131   5606.40              81.20           32.00               03:05   

     duration average midsleep average awake time  
id                                                 
101     10.99            02:32              08:00  
102     13.03            05:00              11:00  
105      9.54            05:10              09:50  
107      8.41            06:30              10:30  
108      8.40            06:00              11:00  
109      6.89            04:14              07:14  
110     10.61            07:30              13:00  
111      9.40            05:10              09:31  
112      8.16            06:30              10:30  
113      7.85            05:19              09:10  
115      7.29            05:10              08:30  
116      8.87            06:00              10:47  
117      6.82            05:48              09:10  
118      9.36            04:00              08:45  
119      7.72            06:00              09:40  
121      9.03            05:15              09:31  
122      7.69            04:23              08:11  
123      8.56            04:40              09:00  
124      8.17            05:20              09:29  
125      7.91            04:19              08:09  
126      9.35            05:49              10:20  
127      7.53            04:00              08:00  
129      7.72            06:47              10:23  
130      9.74            04:19              08:55  
131      9.20            07:57              12:10

### creating sleep duration difference (row are: non-exams - exams, midweek - weekend)

In [23]:
#seperating working df into weekend measurements and midweek measurements
weekend_df = working_df[working_df['weekend'] == 'Yes'].dropna()
midweek_df = working_df[working_df['weekend'] == 'No'].dropna()
# calculating sum of sleeping hours for each user 
sum_sleep_hours_weekend_df = weekend_df.groupby('id')['duration'].sum().reset_index()
sum_sleep_hours_weekend_df.rename(columns={'duration':'weekend sleep duration'}, inplace=True)
sum_sleep_hours_midweek_df = midweek_df.groupby('id')['duration'].sum().reset_index()
sum_sleep_hours_midweek_df.rename(columns={'duration':'midweek sleep duration'}, inplace=True)
#merging both midweek and weekend tables
sum_of_sleeps_weekend_midweek_df = pd.merge(sum_sleep_hours_weekend_df,sum_sleep_hours_midweek_df, on = "id", how='outer')

#seperating working df into weekend measurements and midweek measurements
exams_period_df = working_df[working_df['exam period'] == 'Yes'].dropna()
non_exams_period_df = working_df[working_df['exam period'] == 'No'].dropna()
# calculating sum of sleeping hours for each user
sum_sleep_hours_exams_df = exams_period_df.groupby('id')['duration'].sum().reset_index()
sum_sleep_hours_exams_df.rename(columns={'duration':'exams sleep duration'}, inplace=True)
sum_sleep_hours_non_exams_df = non_exams_period_df.groupby('id')['duration'].sum().reset_index()
sum_sleep_hours_non_exams_df.rename(columns={'duration':'non-exams sleep duration'}, inplace=True)
#merging both exams and non-exams tables
sum_of_sleeps_exams_non_exams_df = pd.merge(sum_sleep_hours_exams_df,sum_sleep_hours_non_exams_df, on = "id", how='outer')

#merging all sleeping duration tables
sleeps_manipulations_df = pd.merge(sum_of_sleeps_weekend_midweek_df, sum_of_sleeps_exams_non_exams_df, on='id', how='outer')
sleeps_manipulations_df.fillna(0, inplace=True)

#calculating sleep diffrence between (midweek - weekend) and (exams - non exams)
sleeps_manipulations_df['midweek - weekend'] = sleeps_manipulations_df['midweek sleep duration'] - sleeps_manipulations_df['weekend sleep duration']
sleeps_manipulations_df['non exams - exams'] = sleeps_manipulations_df['non-exams sleep duration'] - sleeps_manipulations_df['exams sleep duration']

#counting sleeping measures for each student
count_sleep_hours_weekend_df = weekend_df.groupby('id')['duration'].count().reset_index()
count_sleep_hours_weekend_df.rename(columns={'duration':'weekend measurements count'}, inplace=True)
count_sleep_hours_midweek_df = midweek_df.groupby('id')['duration'].count().reset_index()
count_sleep_hours_midweek_df.rename(columns={'duration':'midweek measurements count'}, inplace=True)
count_sleep_hours_exams_df = exams_period_df.groupby('id')['duration'].count().reset_index()
count_sleep_hours_exams_df.rename(columns={'duration':'exams measurements count'}, inplace=True)
count_sleep_hours_non_exams_df = non_exams_period_df.groupby('id')['duration'].count().reset_index()
count_sleep_hours_non_exams_df.rename(columns={'duration':'non-exams measurements count'}, inplace=True)

#merging all counters df
sleeps_counters_df = [count_sleep_hours_weekend_df, count_sleep_hours_midweek_df, count_sleep_hours_exams_df, count_sleep_hours_non_exams_df]
merged_sleeps_counter_df = reduce(lambda left, right: pd.merge(left, right, on="id", how="outer"), sleeps_counters_df)
merged_sleeps_counter_df.fillna(0, inplace=True)

### saving sleeps measurements sum and diffrence dataframe to csv file

In [24]:
sleeps_manipulations_df.to_csv('sleeping-duration-sum-difference.csv')
sleeps_manipulations_df

id  weekend sleep duration  midweek sleep duration  exams sleep duration  \
0   101                  174.97                  386.64                209.56   
1   102                   13.03                    0.00                 13.03   
2   105                  161.31                  380.46                198.99   
3   107                  151.04                  468.76                135.63   
4   108                   42.35                  210.18                 99.26   
5   109                   58.39                  173.93                100.83   
6   110                   59.67                  125.60                 48.56   
7   111                  149.02                  420.86                210.22   
8   112                   72.04                  110.76                 86.63   
9   113                  143.64                  303.81                120.88   
10  115                  118.33                  345.96                172.01   
11  116                  163.34                  409.32                198.72   
12  117                   90.26                  270.67                144.98   
13  118                  129.21                  346.55                162.92   
14  119                  134.20                  314.08                165.35   
15  121                  126.76                  386.13                180.05   
16  122                  113.22                  239.61                131.49   
17  123                  140.42                  387.16                194.43   
18  124                  138.09                  312.69                177.40   
19  125                  132.28                  353.80                177.27   
20  126                  108.14                  316.05                197.81   
21  127                   81.16                  220.08                137.03   
22  129                  110.46                  221.79                121.97   
23  130                  118.66                  235.92                118.88   
24  131                   94.84                  151.64                 82.82   
25  201                  141.98                  370.09                149.87   
26  202                  149.26                  359.81                177.11   
27  203                  104.30                  226.17                 87.69   
28  204                   18.29                   42.32                 60.61   
29  205                  122.09                  286.65                131.19   
30  206                  104.57                  300.90                157.16   
31  207                   78.82                  257.22                118.95   
32  208                  119.14                  285.89                148.04   
33  209                  127.41                  263.16                128.21   
34  210                   96.89                  275.29                119.73   
35  211                   56.65                  142.80                 94.18   
36  213                  153.92                  360.19                203.70   
37  214                  143.70                  293.06                164.23   
38  215                  129.64                  271.40                141.32   
39  216                   98.25                  216.78                 93.05   
40  217                  101.95                  237.75                123.79   
41  218                   54.45                  119.85                 33.90   
42  219                  115.54                  335.07                133.58   
43  220                   34.61                  208.56                119.11   
44  221                  145.97                  336.18                171.64   
45  222                   56.17                  238.37                 95.08   
46  223                  165.18                  386.36                203.07   
47  224                   57.46                  197.10                126.24   
48  225                   99.21     

### saving sleeping measurements count dataframe to csv file

In [25]:
merged_sleeps_counter_df.to_csv('sleeping_measurements_count.csv')
merged_sleeps_counter_df

id  weekend measurements count  midweek measurements count  \
0   101                          18                        45.0   
1   102                           1                         0.0   
2   105                          18                        43.0   
3   107                          14                        46.0   
4   108                           5                        26.0   
5   109                           8                        24.0   
6   110                           7                        14.0   
7   111                          17                        45.0   
8   112                           9                        14.0   
9   113                          17                        39.0   
10  115                          17                        45.0   
11  116                          18                        45.0   
12  117                          12                        34.0   
13  118                          15                        43.0   
14  119                          18                        43.0   
15  121                          16                        45.0   
16  122                          14                        31.0   
17  123                          17                        44.0   
18  124                          17                        40.0   
19  125                          16                        45.0   
20  126                          12                        35.0   
21  127                          10                        27.0   
22  129                          15                        29.0   
23  130                          11                        26.0   
24  131                          10                        16.0   
25  201                          13                        39.0   
26  202                          17                        45.0   
27  203                          13                        30.0   
28  204                           2                         5.0   
29  205                          14                        34.0   
30  206                          14                        42.0   
31  207                          10                        37.0   
32  208                          13                        34.0   
33  209                          15                        33.0   
34  210                          14                        40.0   
35  211                           7                        19.0   
36  213                          17                        40.0   
37  214                          17                        41.0   
38  215                          15                        42.0   
39  216                          13                        29.0   
40  217                          15                        36.0   
41  218                           6                        13.0   
42  219                          14                        42.0   
43  220                           5                        26.0   
44  221                          17                        44.0   
45  222                           7                        31.0   
46  223                          18                        43.0   
47  224                           7                        25.0   
48  225                          14                        34.0   
49  227                          17                        43.0   
50  228                          14                        35.0   
51  229                          12                        25.0   

    exams measurements count  non-exams measurements count  
0                         22                          41.0  
1                          1                           0.0  
2                         22                          39.0  
3                         14                          46.0  
4                         12                          19.0  
5                         15                          17.0  
6                         

#### Merging all averages tables

In [26]:
#adding description columns to all averages dataframes
morning_exams_midweek_average_df["type"] = ['Morning' for i in range(len(morning_exams_midweek_average_df))]
morning_exams_midweek_average_df["exams period"] = ['Yes' for i in range(len(morning_exams_midweek_average_df))]
morning_exams_midweek_average_df["weekend"] = ['No' for i in range(len(morning_exams_midweek_average_df))]

morning_exams_weekend_average_df["type"] = ['Morning' for i in range(len(morning_exams_weekend_average_df))]
morning_exams_weekend_average_df["exams period"] = ['Yes' for i in range(len(morning_exams_weekend_average_df))]
morning_exams_weekend_average_df["weekend"] = ['Yes' for i in range(len(morning_exams_weekend_average_df))]

morning_non_exams_weekend_averages_df["type"] = ['Morning' for i in range(len(morning_non_exams_weekend_averages_df))]
morning_non_exams_weekend_averages_df["exams period"] = ['No' for i in range(len(morning_non_exams_weekend_averages_df))]
morning_non_exams_weekend_averages_df["weekend"] = ['Yes' for i in range(len(morning_non_exams_weekend_averages_df))]

morning_non_exams_midweek_average_df["type"] = ['Morning' for i in range(len(morning_non_exams_midweek_average_df))]
morning_non_exams_midweek_average_df["exams period"] = ['No' for i in range(len(morning_non_exams_midweek_average_df))]
morning_non_exams_midweek_average_df["weekend"] = ['No' for i in range(len(morning_non_exams_midweek_average_df))]

night_non_exams_midweek_average_df["type"] = ['Night' for i in range(len(night_non_exams_midweek_average_df))]
night_non_exams_midweek_average_df["exams period"] = ['No' for i in range(len(night_non_exams_midweek_average_df))]
night_non_exams_midweek_average_df["weekend"] = ['No' for i in range(len(night_non_exams_midweek_average_df))]

night_exams_midweek_average_df["type"] = ['Night' for i in range(len(night_exams_midweek_average_df))]
night_exams_midweek_average_df["exams period"] = ['Yes' for i in range(len(night_exams_midweek_average_df))]
night_exams_midweek_average_df["weekend"] = ['No' for i in range(len(night_exams_midweek_average_df))]

night_non_exams_weekend_average_df["type"] = ['Night' for i in range(len(night_non_exams_weekend_average_df))]
night_non_exams_weekend_average_df["exams period"] = ['No' for i in range(len(night_non_exams_weekend_average_df))]
night_non_exams_weekend_average_df["weekend"] = ['Yes' for i in range(len(night_non_exams_weekend_average_df))]

night_exams_weekend_average_df["type"] = ['Night' for i in range(len(night_exams_weekend_average_df))]
night_exams_weekend_average_df["exams period"] = ['Yes' for i in range(len(night_exams_weekend_average_df))]
night_exams_weekend_average_df["weekend"] = ['Yes' for i in range(len(night_exams_weekend_average_df))]

merged_averages_df = pd.concat([morning_exams_midweek_average_df, morning_exams_weekend_average_df, morning_non_exams_weekend_averages_df,
                                morning_non_exams_midweek_average_df, night_non_exams_midweek_average_df, night_exams_midweek_average_df,
                                night_non_exams_weekend_average_df, night_exams_weekend_average_df]).reset_index()
sorted_by_id_merged_averages_df = merged_averages_df.sort_values('id').set_index('id').reset_index()

### saving merged averages dataframe to csv file

In [27]:
merged_averages_df.to_csv('merged_averages.csv')
sorted_by_id_merged_averages_df.to_csv('sorted_merged_averages.csv')
sorted_by_id_merged_averages_df

id     steps  average heartrate  average stress average sleep start  \
0    101   4958.88              72.31           29.00               22:37   
1    101   6443.00              74.75           28.65               23:05   
2    101   3860.24              74.44           22.07               23:02   
3    101   3392.33              66.33           18.50               21:12   
4    102  11666.90              95.00           36.67                 NaN   
..   ...       ...                ...             ...                 ...   
203  228   5689.00              73.50           27.33               22:47   
204  229   3499.50              82.67           33.67               23:40   
205  229   4954.00              84.71           40.14               23:19   
206  229   5611.89              80.00           26.38               23:58   
207  229   6519.52              80.79           32.54               22:33   

     duration average midsleep average awake time     type exams period  \
0        8.98            03:03              07:29    Night          Yes   
1        8.38            03:16              07:29    Night           No   
2        9.09            03:41              07:55    Night           No   
3       10.99            02:32              08:00    Night          Yes   
4         NaN              NaN                NaN    Night           No   
..        ...              ...                ...      ...          ...   
203      7.48            02:35              06:36  Morning          Yes   
204      7.95            03:40              07:40  Morning          Yes   
205      7.65            03:15              06:59  Morning          Yes   
206      7.09            03:31              07:05  Morning           No   
207      7.95            02:31              06:40  Morning           No   

    weekend  
0        No  
1        No  
2       Yes  
3       Yes  
4       Yes  
..      ...  
203     Yes  
204     Yes  
205      No  
206     Yes  
207      No  

[208 rows x 11 columns]

### creating average sleep duration table for each tested student

In [28]:
averages_df = create_avg_table_from_df(working_df).reset_index()
sleep_duration_averages_df = averages_df[['id','duration']].set_index('id')

In [29]:
sleep_duration_averages_df.to_csv('sleep_duration_averages.csv')
sleep_duration_averages_df

duration
id           
101      8.91
102     13.03
105      8.88
107     10.33
108      8.15
109      7.26
110      8.82
111      9.19
112      7.95
113      7.99
115      7.49
116      9.09
117      7.85
118      8.20
119      7.35
121      8.41
122      7.84
123      8.65
124      7.91
125      7.97
126      9.03
127      8.14
129      7.55
130      9.58
131      9.48
201      9.85
202      8.21
203      7.69
204      8.66
205      8.52
206      7.24
207      7.15
208      8.62
209      8.14
210      6.89
211      7.67
213      9.02
214      7.53
215      7.04
216      7.50
217      6.66
218      9.17
219      8.05
220      7.84
221      7.90
222      7.75
223      9.04
224      7.96
225      7.58
227      8.41
228      7.27
229      7.71

### creating average midsleep table for midweek measurements

In [30]:
mid_week_df = working_df[ (working_df['weekend']=='No')]
mid_week_averages_df = create_avg_table_from_df(mid_week_df).reset_index()
mid_week_midsleep_averages_df = mid_week_averages_df[['id','average midsleep']].set_index('id')

In [31]:
mid_week_midsleep_averages_df.to_csv('midweek_midsleep_average.csv')
mid_week_midsleep_averages_df

average midsleep
id                  
101            03:11
102              NaN
105            05:04
107            05:17
108            04:20
109            04:00
110            07:31
111            04:26
112            05:00
113            04:58
115            04:27
116            05:05
117            04:11
118            03:59
119            04:59
121            04:33
122            03:36
123            05:02
124            05:36
125            04:09
126            04:53
127            04:11
129            05:11
130            03:36
131            06:43
201            03:08
202            03:00
203            01:59
204            03:36
205            03:10
206            03:22
207            03:20
208            03:49
209            03:06
210            03:19
211            02:57
213            03:22
214            03:07
215            03:02
216            04:00
217            03:07
218            02:55
219            02:45
220            02:15
221            03:05
222            02:40
223            01:06
224            02:52
225            02:57
227            03:58
228            03:36
229            02:53

### creating average midsleep table for weekend measurements

In [32]:
weekend_df = working_df[ (working_df['weekend']=='Yes')]
weekend_averages_df = create_avg_table_from_df(weekend_df).reset_index()
weekend_midsleep_averages_df = weekend_averages_df[['id','average midsleep']].set_index('id')

In [33]:
weekend_midsleep_averages_df.to_csv('weekend_midsleep_average.csv')
weekend_midsleep_averages_df

average midsleep
id                  
101            03:20
102            05:00
105            04:56
107            06:07
108            05:48
109            04:37
110            05:51
111            04:39
112            06:47
113            05:58
115            05:51
116            06:29
117            06:02
118            04:35
119            06:10
121            04:25
122            04:23
123            05:23
124            05:20
125            04:35
126            05:45
127            04:23
129            06:48
130            03:38
131            08:34
201            04:18
202            03:59
203            02:09
204            03:29
205            03:30
206            04:09
207            04:42
208            04:18
209            03:23
210            03:47
211            03:08
213            03:59
214            04:10
215            04:03
216            04:41
217            03:28
218            02:59
219            03:21
220            02:53
221            03:10
222            02:59
223            01:39
224            02:51
225            04:51
227            05:00
228            04:32
229            03:35